In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import SimpleRNN, LSTM
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer as imputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import glob

In [2]:
df_list = []
labels = []

for f in glob.glob("training/" + "/*.psv"):
    df_list.append(pd.read_csv(f,sep='|'))
    labels.append(df_list[len(df_list)-1]["SepsisLabel"])
    
    

for i in df_list:
    i.drop('SepsisLabel', axis=1, inplace=True)
    
# This part will be used to get test data, however since there is a dimension problem
# e.g. 1st patch is 39 after imputation and 2nd one is 40, we will work with this later on
# x_test = []
# y_test = []

# for f in glob.glob("training_setB/" + "/*.psv"):
#     x_test.append(pd.read_csv(f,sep='|'))
#     y_test.append(x_test[len(x_test)-1]["SepsisLabel"])
    
    
# for i in x_test:
#     i.drop('SepsisLabel', axis=1 , inplace=True)

In [4]:
#This function returns an array where each patient has their final label.
def convert_label(set_of_labels):
    labels = []
    
    for i in set_of_labels:
        labels.append(np.max(i))
        
    return np.array(labels)

# #This function returns an mean of a list w.r.t. each column.
# def get_mean(theList):
#     result = []
    
#     for i in theList:
#         for row in i.values:
#             result.append(row)
            
#     result = np.array(result)
#     result = np.nanmean(result.astype('float64'), axis=0)
    
#     for i in result:
#         if np.isnan(i):
#             #dosomething
            
#     return result


# Mean imputer
def impute_mean(theList):
    shape = []
    result = []
    cur = 0
    
    for i in theList:
        shape.append(i.shape[0])
            
    df = pd.concat(theList)
    df.fillna(df.mean())
    
    for i in shape:
        result.append(df.iloc[cur:cur+i,:])
        cur += i
    
    return result

# This functions checks if two lists has the same size and shape    
def check_shape(list1, list2):
    
    counter = 0
    
    for i in range(len(list1)):
        if list1[i].shape[0] != list2[i].shape[0]:
            counter += 1
                
        
    return True if counter == 0 else False

In [5]:
x_train = pad_sequences(impute_mean(df_list), dtype='float32',padding='post')
y_train = pad_sequences(labels, dtype='float32',padding='post')


x_train, x_test, y_train, y_test = train_test_split(x_train,y_train,test_size=0.4)

In [25]:
batch_size = x_train.shape[1]
input_dim = x_train.shape[2]

output_size = x_train.shape[1]


model = Sequential()
# model.add(LSTM(32, input_shape=(336, 40)))
model.add(LSTM(16, input_shape=(x_train.shape[-2:]), activation = 'relu'))
model.add(Dense(336))    

# model.add(Dense(336), activation = 'softmax')   

# Compile the model ready for training
model.compile('SGD', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()





Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 16)                3648      
_________________________________________________________________
dense_3 (Dense)              (None, 336)               5712      
Total params: 9,360
Trainable params: 9,360
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(x_train, y_train, batch_size = batch_size, epochs=1)


Train on 12201 samples
12201/12201 [==============================] - 18s 2ms/sample - loss: nan - accuracy: 0.9975


In [24]:
a = model.predict_classes(x_test[0:2,:,:])

In [9]:
predictions = model.predict(x_test)
print(predictions.shape)

(8135, 336)


In [10]:
predicted_probability = model.predict(x_test)

predicted_class = predicted_classes = model.predict_classes(x_test)

correct_preds = np.nonzero(predicted_classes == y_test)[0]
incorrect_preds = np.nonzero(predicted_classes != y_test)[0]
print("{0} classified correctly".format(len(correct_preds)))
print("{0} classified incorrectly".format(len(incorrect_preds)))
print("{0:.2f}% of test set classified correctly".format(len(correct_preds) / len(y_test) * 100))

0 classified correctly
1 classified incorrectly
0.00% of test set classified correctly


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [149]:
def split_before_sepsis(datalist, labels, target):
    data = []
    
    for i in range(datalist.shape[0]):
        if np.max(labels[i])==1:
            index = np.argmax(np.array(labels[i]))
            if (index-target)<0:
                index = 1
            else:
                index = index - target
            data.append(datalist[i,0:index,:])
        else:
            data.append(datalist[i,:,:])
            
            
    return np.array(data)     

In [150]:
xTest = split_before_sepsis(x_test,y_test,6)


In [199]:
# dataset = tf.data.Dataset.from_generator(lambda: xTest, tf.int32, output_shapes=[None])

dataset = tf.data.Dataset.from_tensor_slices(tf.ragged.constant(xTest))

In [177]:
print(xTest.shape)
print(xTest[9].shape)

model.predict(pad_sequences(xTest, dtype='float32',padding='post'))

# model.predict(np.array(xTest[9],dtype=np.float))

(8135,)
(6, 40)


array([[nan, nan],
       [nan, nan],
       [nan, nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]], dtype=float32)

In [ ]:
for x, y in val_data_multi.take(3):
  multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])